# R et MongoDB - Utilisation avec `R`

Dans ce document, nous utilisons le package [`pymongo`](https://docs.mongodb.com/drivers/pymongo/), permettant la connection à une base de données [MongoDB](https://www.mongodb.com/fr). Il fait suite à [l'introduction à MongoDB](du-abd--slides.html)

Le but de ce document est donc de montré les exemples d'utilisation des différentes commandes permettant la récupération de données sur le serveur.

## Exemples sur `restaurants`

Ici, nous allons nous connecter sur un serveur distant, et travailler sur une base des restaurants New-Yorkais.

```
USER = "user"
PASS = "user"
HOST = "cluster0.ougec.mongodb.net"
URI = sprintf("mongodb+srv://%s:%s@%s/", USER, PASS, HOST)
```


In [1]:
import pymongo
client = pymongo.MongoClient()
db = client.test

Le premier document est présenté ci-dessous. La base contient les informations de plus de 25000 restaurants new-yorkais (base de test fournie par [Mongo](https://docs.mongodb.com/getting-started/shell/import-data/)).

```json
{
        "_id" : ObjectId("58ac16d1a251358ee4ee87de"),
        "address" : {
                "building" : "469",
                "coord" : [
                        -73.961704,
                        40.662942
                ],
                "street" : "Flatbush Avenue",
                "zipcode" : "11225"
        },
        "borough" : "Brooklyn",
        "cuisine" : "Hamburgers",
        "grades" : [
                {
                        "date" : ISODate("2014-12-30T00:00:00Z"),
                        "grade" : "A",
                        "score" : 8
                },
                {
                        "date" : ISODate("2014-07-01T00:00:00Z"),
                        "grade" : "B",
                        "score" : 23
                },
                {
                        "date" : ISODate("2013-04-30T00:00:00Z"),
                        "grade" : "A",
                        "score" : 12
                },
                {
                        "date" : ISODate("2012-05-08T00:00:00Z"),
                        "grade" : "A",
                        "score" : 12
                }
        ],
        "name" : "Wendy'S",
        "restaurant_id" : "30112340"
}
```

### Document dans `python`

Les données `JSON` sont similaires à un dictionnaire `python`. Pour récupérer le premier document, nous utilisons la fonction `find()` de l'objet créé `m`.

In [5]:
d = db.restaurants.find(limit = 1)
d

L'objet retourné est un **curseur**, et non le résultat. Nous avons celui-ci lorsque nous utilisons `d` dans une commande telle qu'une transformation en `list` par exemple. Une fois le résultat retourné (un seul élément ici), le curseur ne renvoie plus rien

In [6]:
list(d)

[{'_id': ObjectId('60006d6aa7aafd5a6d45ca9a'),
  'address': {'building': '1007',
   'coord': [-73.856077, 40.848447],
   'street': 'Morris Park Ave',
   'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
    'grade': 'A',
    'score': 2},
   {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
   {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
   {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
   {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'}]

In [7]:
list(d)

[]

### Dénombrement

Comme indiqué ci-dessous, on utilise la fonction `count_documents({})` pour dénombrer les documents. La fonction `estimated_document_count()` permet elle d'estimer le nombre de documents. Elle est intéressante dans le cas de multiples serveurs et de données massives.

- Tous les restaurants

In [10]:
db.restaurants.count_documents({})

25359

In [14]:
db.restaurants.estimated_document_count()

25359

- Restaurants de *Brooklyn*

In [19]:
db.restaurants.count_documents({ "borough": "Brooklyn" })

6086

- Restaurants de *Brooklyn* proposant de la cuisine française

In [20]:
db.restaurants.count_documents({ "borough": "Brooklyn", "cuisine": "French" })

54

- Restaurants de *Brooklyn* proposant de la cuisine française ou italienne

In [21]:
db.restaurants.count_documents({ "borough": "Brooklyn", "cuisine": { "$in": ["French", "Italian"]} })

246

- Idem mais écrit plus lisiblement

In [22]:
db.restaurants.count_documents(
  { 
    "borough": "Brooklyn", 
    "cuisine": { "$in": ["French", "Italian"]}
  }
)

246

- Restaurants situés sur *Franklin Street*
    - Notez l'accès au champs `street` du champs `address`

In [23]:
db.restaurants.count_documents(
  { 
    "address.street": "Franklin Street"
  }
)

25

- Restaurants ayant eu un score de 0

In [24]:
db.restaurants.count_documents(
  { 
    "grades.score": 0
  }
)

1246

- Restaurants ayant eu un score inférieur à 5

In [25]:
db.restaurants.count_documents(
  { 
    "grades.score": { "$lte": 5 }
  }
)

10650

### Valeurs distinctes

On peut aussi voir la liste des valeurs distinctes d'un attribut, avec la fonction `distinct()`.

- Quartier (`borough`), pour tous les restaurants

In [26]:
db.restaurants.distinct(key = "borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Missing', 'Queens', 'Staten Island']

- Cuisine pour les restaurants de *Brooklyn*

In [28]:
db.restaurants.distinct(
  key = "cuisine",
  query = { "borough": "Brooklyn" }
)

['Afghan',
 'African',
 'American ',
 'Armenian',
 'Asian',
 'Australian',
 'Bagels/Pretzels',
 'Bakery',
 'Bangladeshi',
 'Barbecue',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Brazilian',
 'Café/Coffee/Tea',
 'Cajun',
 'Caribbean',
 'Chicken',
 'Chilean',
 'Chinese',
 'Chinese/Cuban',
 'Chinese/Japanese',
 'Continental',
 'Creole',
 'Creole/Cajun',
 'Czech',
 'Delicatessen',
 'Donuts',
 'Eastern European',
 'Egyptian',
 'English',
 'Ethiopian',
 'Filipino',
 'French',
 'Fruits/Vegetables',
 'German',
 'Greek',
 'Hamburgers',
 'Hawaiian',
 'Hotdogs',
 'Hotdogs/Pretzels',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Indian',
 'Indonesian',
 'Irish',
 'Italian',
 'Japanese',
 'Jewish/Kosher',
 'Juice, Smoothies, Fruit Salads',
 'Korean',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Moroccan',
 'Not Listed/Not Applicable',
 'Nuts/Confectionary',
 'Other',
 'Pakistani',
 'Pancakes/Waffles',
 'Peruvian',

- Grade des restaurants de *Brooklyn*

In [29]:
db.restaurants.distinct(
  key = "grades.grade",
  query = { "borough": "Brooklyn" }
)

['A', 'B', 'C', 'Not Yet Graded', 'P', 'Z']

### Restriction et Projection

La fonction `find()` de l'objet `m` permet de réaliser les *restrictions* et *projections*. Elle prend deux paramètres : la restriction à faire et quels champs afficher. Elle renvoie un curseur, qu'il faut donc gérer pour avoir le résultat. Ici, nous transformons les données en `DataFrame` (du module `pandas`). On verra que ce format n'est pas forcément idéal pour certains champs, mais permet un affichage propre du résultat.

In [50]:
import pandas

- Restaurants *Shake Shack* (uniquement les attributs `"street"` et `"borough"`)

In [51]:
c = db.restaurants.find({ "name": "Shake Shack" }, { "address.street": 1, "borough": 1 })
pandas.DataFrame(c)

,_id,address,borough
0,60006d6ca7aafd5a6d45ec73,{'street': 'Columbus Avenue'},Manhattan
1,60006d6ca7aafd5a6d45f66d,{'street': 'West 44 Street'},Manhattan
2,60006d6ca7aafd5a6d45f66e,{'street': 'East 86 Street'},Manhattan
3,60006d6ca7aafd5a6d45fe35,{'street': 'North End Avenue'},Manhattan
4,60006d6ca7aafd5a6d45ff7f,{'street': 'Fulton Street'},Brooklyn
5,60006d6da7aafd5a6d461437,{'street': 'Jfk International Airport'},Queens
6,60006d6da7aafd5a6d4618f1,{'street': 'Grand Central Terminal'},Manhattan
7,60006d6da7aafd5a6d46197c,{'street': 'Jfk International Airport'},Queens
8,60006d6ea7aafd5a6d462178,{'street': 'Old Fulton Street'},Brooklyn
9,60006d6ea7aafd5a6d46217a,{'street': 'Flatbush Avenue'},Brooklyn


- Idem sans l'identifiant interne

In [52]:
c = db.restaurants.find(
    { "name": "Shake Shack" }, 
    { "_id": 0, "address.street": 1, "borough": 1 }
)
pandas.DataFrame(c)

,address,borough
0,{'street': 'Columbus Avenue'},Manhattan
1,{'street': 'West 44 Street'},Manhattan
2,{'street': 'East 86 Street'},Manhattan
3,{'street': 'North End Avenue'},Manhattan
4,{'street': 'Fulton Street'},Brooklyn
5,{'street': 'Jfk International Airport'},Queens
6,{'street': 'Grand Central Terminal'},Manhattan
7,{'street': 'Jfk International Airport'},Queens
8,{'street': 'Old Fulton Street'},Brooklyn
9,{'street': 'Flatbush Avenue'},Brooklyn


- 5 premiers restaurants du quartier *Queens*, avec une note A et un score supérieur à 50 (on affiche le nom et la rue du restaurant

In [53]:
c = db.restaurants.find(
    {"borough": "Queens", "grades.score": { "$gte":  50}},
    {"_id": 0, "name": 1, "grades.score": 1, "address.street": 1},
    limit = 5
)
pandas.DataFrame(c)

,address,grades,name
0,{'street': 'Horace Harding Boulevard'},"[{'score': 12}, {'score': 4}, {'score': 11}, {...",Richer'S Bakery
1,{'street': 'Bell Boulevard'},"[{'score': 52}, {'score': 12}, {'score': 22}, ...",Tequilla Sunrise
2,{'street': 'Rockaway Beach Boulevard'},"[{'score': 10}, {'score': 2}, {'score': 10}, {...",Rockaway Beach Inn
3,{'street': 'Broadway'},"[{'score': 13}, {'score': 13}, {'score': 13}, ...",Alfonso'S Bar
4,{'street': 'Woodhaven Boulevard'},"[{'score': 2}, {'score': 64}, {'score': 9}, {'...",Pio Pio


- Restaurants *Shake Shack* dans différents quartiers (*Queens* et *Brooklyn*)

In [54]:
c = db.restaurants.find(
    {"name": "Shake Shack", "borough": {"$in": ["Queens", "Brooklyn"]}}, 
    {"_id": 0, "address.street": 1, "borough": 1}
)
pandas.DataFrame(c)

,address,borough
0,{'street': 'Fulton Street'},Brooklyn
1,{'street': 'Jfk International Airport'},Queens
2,{'street': 'Jfk International Airport'},Queens
3,{'street': 'Old Fulton Street'},Brooklyn
4,{'street': 'Flatbush Avenue'},Brooklyn


- Restaurants du Queens ayant une note supérieure à 50, mais trié par ordre décroissant de noms de rue, et ordre croissant de noms de restaurants
    - Pour le tri, on doit passer un tuple, composé de tuples à 2 valeurs : le champs de tri et l'ordre (1 croissant et -1 décroissant)

In [55]:
c = db.restaurants.find(
    {"borough": "Queens", "grades.score": { "$gt":  50}},
    {"_id": 0, "name": 1, "address.street": 1},
    sort = (("address.street", -1), ("name", 1))
)
pandas.DataFrame(c)

,address,name
0,{'street': 'Woodward Avenue'},Sabores Restaurant & Bar
1,{'street': 'Woodside Avenue'},Salza Pizza
2,{'street': 'Woodside Avenue'},Spicy Shallot
3,{'street': 'Woodhaven Boulevard'},Fresh To You
4,{'street': 'Woodhaven Boulevard'},Pio Pio
...,...,...
73,{'street': '30 Avenue'},Queens Comfort Restaurant
74,{'street': '20 Avenue'},Cafeteria (Usps Bldng)
75,{'street': '153 Avenue'},Tuscany Deli
76,{'street': '131 Street'},Spa Castle/Juice Farm


### Agrégat

Ils sont réalisés avec la fonction `aggregate()`, qui permet de faire aussi beaucoup d'autres opérations. Idem que `find()`, elle renvoie un curseur qu'il faut donc gérer pour avoir le résultat à l'affichage (ou autre). 

- Limite aux 5 premiers restaurants

In [56]:
c = db.restaurants.aggregate(
    [
        {"$limit": 10 }
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake Shop,30075445
1,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340
2,60006d6aa7aafd5a6d45ca9c,"{'building': '351', 'coord': [-73.985135599999...",Manhattan,Irish,"[{'date': 2014-09-06 00:00:00, 'grade': 'A', '...",Dj Reynolds Pub And Restaurant,30191841
3,60006d6aa7aafd5a6d45ca9d,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018
4,60006d6aa7aafd5a6d45ca9e,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,"[{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...",Tov Kosher Kitchen,40356068
5,60006d6aa7aafd5a6d45ca9f,"{'building': '8825', 'coord': [-73.8803827, 40...",Queens,American,"[{'date': 2014-11-15 00:00:00, 'grade': 'Z', '...",Brunos On The Boulevard,40356151
6,60006d6aa7aafd5a6d45caa0,"{'building': '2206', 'coord': [-74.1377286, 40...",Staten Island,Jewish/Kosher,"[{'date': 2014-10-06 00:00:00, 'grade': 'A', '...",Kosher Island,40356442
7,60006d6aa7aafd5a6d45caa1,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483
8,60006d6aa7aafd5a6d45caa2,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649
9,60006d6aa7aafd5a6d45caa3,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731


- Idem avec tri sur le nom du restaurant

In [57]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$sort": { "name": 1 }}
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9f,"{'building': '8825', 'coord': [-73.8803827, 40...",Queens,American,"[{'date': 2014-11-15 00:00:00, 'grade': 'Z', '...",Brunos On The Boulevard,40356151
1,60006d6aa7aafd5a6d45ca9c,"{'building': '351', 'coord': [-73.985135599999...",Manhattan,Irish,"[{'date': 2014-09-06 00:00:00, 'grade': 'A', '...",Dj Reynolds Pub And Restaurant,30191841
2,60006d6aa7aafd5a6d45caa0,"{'building': '2206', 'coord': [-74.1377286, 40...",Staten Island,Jewish/Kosher,"[{'date': 2014-10-06 00:00:00, 'grade': 'A', '...",Kosher Island,40356442
3,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake Shop,30075445
4,60006d6aa7aafd5a6d45caa2,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649
5,60006d6aa7aafd5a6d45ca9d,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018
6,60006d6aa7aafd5a6d45caa3,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731
7,60006d6aa7aafd5a6d45ca9e,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,"[{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...",Tov Kosher Kitchen,40356068
8,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340
9,60006d6aa7aafd5a6d45caa1,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483


- Idem en se restreignant à *Brooklyn*
    - Notez que nous obtenons uniquement 5 restaurants au final

In [58]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$sort": { "name": 1 }},
        { "$match": { "borough": "Brooklyn" }}
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45caa2,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649
1,60006d6aa7aafd5a6d45ca9d,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018
2,60006d6aa7aafd5a6d45caa3,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731
3,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340
4,60006d6aa7aafd5a6d45caa1,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483


- Mêmes opérations mais avec la restriction en amont de la limite
    - Nous avons ici les 10 premiers restaurants de *Brooklyn* donc

In [59]:
c = db.restaurants.aggregate(
    [
        { "$match": { "borough": "Brooklyn" }},
        { "$limit": 10 },
        { "$sort": { "name": 1 }}
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45caa5,"{'building': '7715', 'coord': [-73.9973325, 40...",Brooklyn,American,"[{'date': 2014-04-16 00:00:00, 'grade': 'A', '...",C & C Catering Service,40357437
1,60006d6aa7aafd5a6d45caa9,"{'building': '203', 'coord': [-73.978220400000...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-02-10 00:00:00, 'grade': 'A', '...",Carvel Ice Cream,40360076
2,60006d6aa7aafd5a6d45caa6,"{'building': '1269', 'coord': [-73.871194, 40....",Brooklyn,Chinese,"[{'date': 2014-09-16 00:00:00, 'grade': 'B', '...",May May Kitchen,40358429
3,60006d6aa7aafd5a6d45caab,"{'building': '6909', 'coord': [-74.0259567, 40...",Brooklyn,Delicatessen,"[{'date': 2014-08-21 00:00:00, 'grade': 'A', '...",Nordic Delicacies,40361390
4,60006d6aa7aafd5a6d45caa2,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649
5,60006d6aa7aafd5a6d45ca9d,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018
6,60006d6aa7aafd5a6d45caa8,"{'building': '705', 'coord': [-73.9653967, 40....",Brooklyn,Jewish/Kosher,"[{'date': 2014-11-10 00:00:00, 'grade': 'A', '...",Seuda Foods,40360045
7,60006d6aa7aafd5a6d45caa3,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731
8,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340
9,60006d6aa7aafd5a6d45caa1,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483


- Séparation des 5 premiers restaurants sur la base des évaluations (`grades`)
    - Chaque ligne correspond maintenant a une évaluation pour un restaurant

In [61]:
c = db.restaurants.aggregate(
    [
        { "$limit": 5 },
        { "$unwind": "$grades" }
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2014-03-03 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
1,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2013-09-11 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
2,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2013-01-24 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
3,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2011-11-23 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
4,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2011-03-10 00:00:00, 'grade': 'B', 's...",Morris Park Bake Shop,30075445
5,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2014-12-30 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
6,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2014-07-01 00:00:00, 'grade': 'B', 's...",Wendy'S,30112340
7,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2013-04-30 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
8,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2012-05-08 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
9,60006d6aa7aafd5a6d45ca9c,"{'building': '351', 'coord': [-73.985135599999...",Manhattan,Irish,"{'date': 2014-09-06 00:00:00, 'grade': 'A', 's...",Dj Reynolds Pub And Restaurant,30191841


- Idem précédemment, en se restreignant à celle ayant eu *B*

In [62]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$unwind": "$grades" },
        { "$match": { "grades.grade": "B" }}
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2011-03-10 00:00:00, 'grade': 'B', 's...",Morris Park Bake Shop,30075445
1,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2014-07-01 00:00:00, 'grade': 'B', 's...",Wendy'S,30112340
2,60006d6aa7aafd5a6d45ca9e,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,"{'date': 2011-12-15 00:00:00, 'grade': 'B', 's...",Tov Kosher Kitchen,40356068


- Si on inverse les opérations `$unwind` et `$match`, le résultat est clairement différent

In [63]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$match": { "grades.grade": "B" }},
        { "$unwind": "$grades" }
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2014-03-03 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
1,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2013-09-11 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
2,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2013-01-24 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
3,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2011-11-23 00:00:00, 'grade': 'A', 's...",Morris Park Bake Shop,30075445
4,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"{'date': 2011-03-10 00:00:00, 'grade': 'B', 's...",Morris Park Bake Shop,30075445
5,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2014-12-30 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
6,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2014-07-01 00:00:00, 'grade': 'B', 's...",Wendy'S,30112340
7,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2013-04-30 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
8,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"{'date': 2012-05-08 00:00:00, 'grade': 'A', 's...",Wendy'S,30112340
9,60006d6aa7aafd5a6d45ca9e,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,"{'date': 2014-11-24 00:00:00, 'grade': 'Z', 's...",Tov Kosher Kitchen,40356068


- On souhaite ici ne garder que le nom et le quartier des 10 premiers restaurants
    - Notez l'ordre (alphabétique) des variables, et pas celui de la déclaration

In [64]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "name": 1, "borough": 1 } }
    ]
)
pandas.DataFrame(c)

,_id,borough,name
0,60006d6aa7aafd5a6d45ca9a,Bronx,Morris Park Bake Shop
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,Wendy'S
2,60006d6aa7aafd5a6d45ca9c,Manhattan,Dj Reynolds Pub And Restaurant
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,Riviera Caterer
4,60006d6aa7aafd5a6d45ca9e,Queens,Tov Kosher Kitchen
5,60006d6aa7aafd5a6d45ca9f,Queens,Brunos On The Boulevard
6,60006d6aa7aafd5a6d45caa0,Staten Island,Kosher Island
7,60006d6aa7aafd5a6d45caa1,Brooklyn,Wilken'S Fine Food
8,60006d6aa7aafd5a6d45caa2,Brooklyn,Regina Caterers
9,60006d6aa7aafd5a6d45caa3,Brooklyn,Taste The Tropics Ice Cream


- Ici, on supprime l'adresse et les évaluations 

In [65]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "address": 0, "grades": 0 } }
    ]
)
pandas.DataFrame(c)

,_id,borough,cuisine,name,restaurant_id
0,60006d6aa7aafd5a6d45ca9a,Bronx,Bakery,Morris Park Bake Shop,30075445
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,Hamburgers,Wendy'S,30112340
2,60006d6aa7aafd5a6d45ca9c,Manhattan,Irish,Dj Reynolds Pub And Restaurant,30191841
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,American,Riviera Caterer,40356018
4,60006d6aa7aafd5a6d45ca9e,Queens,Jewish/Kosher,Tov Kosher Kitchen,40356068
5,60006d6aa7aafd5a6d45ca9f,Queens,American,Brunos On The Boulevard,40356151
6,60006d6aa7aafd5a6d45caa0,Staten Island,Jewish/Kosher,Kosher Island,40356442
7,60006d6aa7aafd5a6d45caa1,Brooklyn,Delicatessen,Wilken'S Fine Food,40356483
8,60006d6aa7aafd5a6d45caa2,Brooklyn,American,Regina Caterers,40356649
9,60006d6aa7aafd5a6d45caa3,Brooklyn,"Ice Cream, Gelato, Yogurt, Ices",Taste The Tropics Ice Cream,40356731


- En plus du nom et du quartier, on récupère l'adresse mais dans un nouveau champs 

In [66]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "name": 1, "borough": 1 , "street": "$address.street"} }
    ]
)
pandas.DataFrame(c)

,_id,borough,name,street
0,60006d6aa7aafd5a6d45ca9a,Bronx,Morris Park Bake Shop,Morris Park Ave
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,Wendy'S,Flatbush Avenue
2,60006d6aa7aafd5a6d45ca9c,Manhattan,Dj Reynolds Pub And Restaurant,West 57 Street
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,Riviera Caterer,Stillwell Avenue
4,60006d6aa7aafd5a6d45ca9e,Queens,Tov Kosher Kitchen,63 Road
5,60006d6aa7aafd5a6d45ca9f,Queens,Brunos On The Boulevard,Astoria Boulevard
6,60006d6aa7aafd5a6d45caa0,Staten Island,Kosher Island,Victory Boulevard
7,60006d6aa7aafd5a6d45caa1,Brooklyn,Wilken'S Fine Food,Avenue U
8,60006d6aa7aafd5a6d45caa2,Brooklyn,Regina Caterers,11 Avenue
9,60006d6aa7aafd5a6d45caa3,Brooklyn,Taste The Tropics Ice Cream,Nostrand Avenue


- On ajoute le nombre de visites pour chaque restaurant (donc la taille du tableau `grades`)

In [67]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "name": 1, "borough": 1, "nb_grades": { "$size": "$grades" } } }
    ]
)
pandas.DataFrame(c)

,_id,borough,name,nb_grades
0,60006d6aa7aafd5a6d45ca9a,Bronx,Morris Park Bake Shop,5
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,Wendy'S,4
2,60006d6aa7aafd5a6d45ca9c,Manhattan,Dj Reynolds Pub And Restaurant,4
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,Riviera Caterer,4
4,60006d6aa7aafd5a6d45ca9e,Queens,Tov Kosher Kitchen,4
5,60006d6aa7aafd5a6d45ca9f,Queens,Brunos On The Boulevard,4
6,60006d6aa7aafd5a6d45caa0,Staten Island,Kosher Island,4
7,60006d6aa7aafd5a6d45caa1,Brooklyn,Wilken'S Fine Food,6
8,60006d6aa7aafd5a6d45caa2,Brooklyn,Regina Caterers,5
9,60006d6aa7aafd5a6d45caa3,Brooklyn,Taste The Tropics Ice Cream,4


- On trie ce résultat par nombre décroissant de visites, et on affiche les 10 premiers

In [73]:
c = db.restaurants.aggregate(
    [
        { "$project": { "name": 1, "borough": 1, "nb_grades": { "$size": "$grades" } } },
        { "$sort": { "nb_grades": -1 }},
        { "$limit": 10 }
    ]
)
pandas.DataFrame(c)

,_id,borough,name,nb_grades
0,60006d6ca7aafd5a6d45f30a,Brooklyn,Silver Krust West Indian Restaurant,9
1,60006d6ba7aafd5a6d45e7c6,Brooklyn,Lai Lai Gourmet,9
2,60006d6da7aafd5a6d46011f,Manhattan,Pure Food,9
3,60006d6ca7aafd5a6d45f9cf,Manhattan,Breeze Thai-French Kitchen,9
4,60006d6ba7aafd5a6d45e35d,Manhattan,Benton,9
5,60006d6ca7aafd5a6d45e9fd,Manhattan,Nomado 33,9
6,60006d6ba7aafd5a6d45e4ce,Brooklyn,Noodle Station,9
7,60006d6ba7aafd5a6d45e332,Manhattan,S'Mac,9
8,60006d6aa7aafd5a6d45ceeb,Manhattan,World Cup Cafe,8
9,60006d6aa7aafd5a6d45cf32,Brooklyn,Fifth Ave Cafe /Diner,8


- On ne garde maintenant que le premier élément du tableau `grades` (indicé 0)

In [74]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "name": 1, "borough": 1, "grade": { "$arrayElemAt": [ "$grades", 0 ]} } }
    ]
)
pandas.DataFrame(c)

,_id,borough,name,grade
0,60006d6aa7aafd5a6d45ca9a,Bronx,Morris Park Bake Shop,"{'date': 2014-03-03 00:00:00, 'grade': 'A', 's..."
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,Wendy'S,"{'date': 2014-12-30 00:00:00, 'grade': 'A', 's..."
2,60006d6aa7aafd5a6d45ca9c,Manhattan,Dj Reynolds Pub And Restaurant,"{'date': 2014-09-06 00:00:00, 'grade': 'A', 's..."
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,Riviera Caterer,"{'date': 2014-06-10 00:00:00, 'grade': 'A', 's..."
4,60006d6aa7aafd5a6d45ca9e,Queens,Tov Kosher Kitchen,"{'date': 2014-11-24 00:00:00, 'grade': 'Z', 's..."
5,60006d6aa7aafd5a6d45ca9f,Queens,Brunos On The Boulevard,"{'date': 2014-11-15 00:00:00, 'grade': 'Z', 's..."
6,60006d6aa7aafd5a6d45caa0,Staten Island,Kosher Island,"{'date': 2014-10-06 00:00:00, 'grade': 'A', 's..."
7,60006d6aa7aafd5a6d45caa1,Brooklyn,Wilken'S Fine Food,"{'date': 2014-05-29 00:00:00, 'grade': 'A', 's..."
8,60006d6aa7aafd5a6d45caa2,Brooklyn,Regina Caterers,"{'date': 2014-07-18 00:00:00, 'grade': 'A', 's..."
9,60006d6aa7aafd5a6d45caa3,Brooklyn,Taste The Tropics Ice Cream,"{'date': 2014-07-14 00:00:00, 'grade': 'A', 's..."


- On peut aussi faire des opérations sur les chaînes, tel que la mise en majuscule du nom

In [75]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { "nom": { "$toUpper": "$name" }, "borough": 1 } }
    ]
)
pandas.DataFrame(c)

,_id,borough,nom
0,60006d6aa7aafd5a6d45ca9a,Bronx,MORRIS PARK BAKE SHOP
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,WENDY'S
2,60006d6aa7aafd5a6d45ca9c,Manhattan,DJ REYNOLDS PUB AND RESTAURANT
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,RIVIERA CATERER
4,60006d6aa7aafd5a6d45ca9e,Queens,TOV KOSHER KITCHEN
5,60006d6aa7aafd5a6d45ca9f,Queens,BRUNOS ON THE BOULEVARD
6,60006d6aa7aafd5a6d45caa0,Staten Island,KOSHER ISLAND
7,60006d6aa7aafd5a6d45caa1,Brooklyn,WILKEN'S FINE FOOD
8,60006d6aa7aafd5a6d45caa2,Brooklyn,REGINA CATERERS
9,60006d6aa7aafd5a6d45caa3,Brooklyn,TASTE THE TROPICS ICE CREAM


- On peut aussi vouloir ajouter un champs, comme ici le nombre d'évaluations

In [76]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$addFields": { "nb_grades": { "$size": "$grades" } } }
    ]
)
pandas.DataFrame(c)

,_id,address,borough,cuisine,grades,name,restaurant_id,nb_grades
0,60006d6aa7aafd5a6d45ca9a,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake Shop,30075445,5
1,60006d6aa7aafd5a6d45ca9b,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340,4
2,60006d6aa7aafd5a6d45ca9c,"{'building': '351', 'coord': [-73.985135599999...",Manhattan,Irish,"[{'date': 2014-09-06 00:00:00, 'grade': 'A', '...",Dj Reynolds Pub And Restaurant,30191841,4
3,60006d6aa7aafd5a6d45ca9d,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018,4
4,60006d6aa7aafd5a6d45ca9e,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,"[{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...",Tov Kosher Kitchen,40356068,4
5,60006d6aa7aafd5a6d45ca9f,"{'building': '8825', 'coord': [-73.8803827, 40...",Queens,American,"[{'date': 2014-11-15 00:00:00, 'grade': 'Z', '...",Brunos On The Boulevard,40356151,4
6,60006d6aa7aafd5a6d45caa0,"{'building': '2206', 'coord': [-74.1377286, 40...",Staten Island,Jewish/Kosher,"[{'date': 2014-10-06 00:00:00, 'grade': 'A', '...",Kosher Island,40356442,4
7,60006d6aa7aafd5a6d45caa1,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483,6
8,60006d6aa7aafd5a6d45caa2,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649,5
9,60006d6aa7aafd5a6d45caa3,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731,4


- On extrait ici les trois premières lettres du quartier

In [77]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$project": { 
            "nom": { "$toUpper": "$name" }, 
            "quartier": { "$substr": [ "$borough", 0, 3 ] } 
        } }
    ]
)
pandas.DataFrame(c)

,_id,nom,quartier
0,60006d6aa7aafd5a6d45ca9a,MORRIS PARK BAKE SHOP,Bro
1,60006d6aa7aafd5a6d45ca9b,WENDY'S,Bro
2,60006d6aa7aafd5a6d45ca9c,DJ REYNOLDS PUB AND RESTAURANT,Man
3,60006d6aa7aafd5a6d45ca9d,RIVIERA CATERER,Bro
4,60006d6aa7aafd5a6d45ca9e,TOV KOSHER KITCHEN,Que
5,60006d6aa7aafd5a6d45ca9f,BRUNOS ON THE BOULEVARD,Que
6,60006d6aa7aafd5a6d45caa0,KOSHER ISLAND,Sta
7,60006d6aa7aafd5a6d45caa1,WILKEN'S FINE FOOD,Bro
8,60006d6aa7aafd5a6d45caa2,REGINA CATERERS,Bro
9,60006d6aa7aafd5a6d45caa3,TASTE THE TROPICS ICE CREAM,Bro


- On fait de même, mais on met en majuscule et on note *BRX* pour le *Bronx*
    - on garde le quartier d'origine pour vérification ici

In [79]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$addFields": { "quartier": { "$toUpper": { "$substr": [ "$borough", 0, 3 ] } } }},
        { "$project": { 
            "nom": { "$toUpper": "$name" }, 
            "quartier": { "$cond": { 
                "if": { "$eq": ["$borough", "Bronx"] }, 
                "then": "BRX", 
                "else": "$quartier" 
            } },
            "borough": 1
        } }
    ]
)
pandas.DataFrame(c)

,_id,borough,nom,quartier
0,60006d6aa7aafd5a6d45ca9a,Bronx,MORRIS PARK BAKE SHOP,BRX
1,60006d6aa7aafd5a6d45ca9b,Brooklyn,WENDY'S,BRO
2,60006d6aa7aafd5a6d45ca9c,Manhattan,DJ REYNOLDS PUB AND RESTAURANT,MAN
3,60006d6aa7aafd5a6d45ca9d,Brooklyn,RIVIERA CATERER,BRO
4,60006d6aa7aafd5a6d45ca9e,Queens,TOV KOSHER KITCHEN,QUE
5,60006d6aa7aafd5a6d45ca9f,Queens,BRUNOS ON THE BOULEVARD,QUE
6,60006d6aa7aafd5a6d45caa0,Staten Island,KOSHER ISLAND,STA
7,60006d6aa7aafd5a6d45caa1,Brooklyn,WILKEN'S FINE FOOD,BRO
8,60006d6aa7aafd5a6d45caa2,Brooklyn,REGINA CATERERS,BRO
9,60006d6aa7aafd5a6d45caa3,Brooklyn,TASTE THE TROPICS ICE CREAM,BRO


- On calcule ici le nombre total de restaurants

In [80]:
c = db.restaurants.aggregate(
    [
        {"$group": {"_id": "Total", "NbRestos": {"$sum": 1}}}
    ]
)
pandas.DataFrame(c)

,_id,NbRestos
0,Total,25359


- On fait de même, mais par quartier

In [81]:
c = db.restaurants.aggregate(
    [
        {"$group": {"_id": "$borough", "NbRestos": {"$sum": 1}}}
    ]
)
pandas.DataFrame(c)

,_id,NbRestos
0,Queens,5656
1,Missing,51
2,Bronx,2338
3,Manhattan,10259
4,Brooklyn,6086
5,Staten Island,969


- Pour faire le calcul des notes moyennes des restaurants du *Queens*, on exécute le code suivant

In [82]:
c = db.restaurants.aggregate(
    [
        { "$match": { "borough": "Queens" }},
        { "$unwind": "$grades" },
        { "$group": { "_id": "null", "score": { "$avg": "$grades.score" }}}
    ]
)
pandas.DataFrame(c)

,_id,score
0,null,11.634865


-  Il est bien évidemment possible de faire ce calcul par quartier et de les trier selon les notes obtenues (dans l'ordre décroissant)

In [83]:
c = db.restaurants.aggregate(
    [
        { "$unwind": "$grades" },
        { "$group": { "_id": "$borough", "score": { "$avg": "$grades.score" }}},
        { "$sort": { "score": -1 }}
    ]
)
pandas.DataFrame(c)

,_id,score
0,Queens,11.634865
1,Brooklyn,11.447976
2,Manhattan,11.418151
3,Staten Island,11.370958
4,Bronx,11.036186
5,Missing,9.632911


- On peut aussi faire un regroupement par quartier et par rue (en ne prenant que la première évaluation - qui est la dernière en date a priori), pour afficher les 10 rues où on mange le plus sainement
    - Notez que le `$match` permet de supprimer les restaurants sans évaluations (ce qui engendrerait des moyennes = `None`)

In [91]:
c = db.restaurants.aggregate(
    [
        { "$project": {
            "borough": 1, "street": "$address.street", 
            "eval": { "$arrayElemAt": [ "$grades", 0 ]} 
        } },
        { "$match": { "eval": { "$exists": True } } },
        { "$group": { 
            "_id": { "quartier": "$borough", "rue": "$street" }, 
            "score": { "$avg": "$eval.score" }
        }},
        { "$sort": { "score": 1 }},
        { "$limit": 10 }
    ]
)
pandas.DataFrame(c)

,_id,score
0,"{'quartier': 'Manhattan', 'rue': '106 Street &...",-1.0
1,"{'quartier': 'Brooklyn', 'rue': '78Th St'}",0.0
2,"{'quartier': 'Manhattan', 'rue': 'Pleasant Ave...",0.0
3,"{'quartier': 'Manhattan', 'rue': 'Oliver St'}",0.0
4,"{'quartier': 'Manhattan', 'rue': 'E Bway'}",0.0
5,"{'quartier': 'Missing', 'rue': 'Clintonville St'}",0.0
6,"{'quartier': 'Manhattan', 'rue': 'Leroy Street'}",0.0
7,"{'quartier': 'Staten Island', 'rue': 'Eton Pla...",0.0
8,"{'quartier': 'Brooklyn', 'rue': 'Stockholm St'}",0.0
9,"{'quartier': 'Bronx', 'rue': 'East 208 Street'}",0.0


- Pour comprendre la différence entre `$addToSet` et `$push`, on les applique sur les grades obtenus pour les 10 premiers restaurants
    - `$addToSet` : valeurs distinctes
    - `$push` : toutes les valeurs présentes

In [92]:
c = db.restaurants.aggregate(
    [
        { "$limit": 10 },
        { "$unwind": "$grades" },
        { "$group": { 
            "_id": "$name", 
            "avec_addToSet": { "$addToSet": "$grades.grade" },
            "avec_push": { "$push": "$grades.grade" }
        }}
    ]
)
pandas.DataFrame(c)

,_id,avec_addToSet,avec_push
0,Dj Reynolds Pub And Restaurant,[A],"[A, A, A, A]"
1,Kosher Island,[A],"[A, A, A, A]"
2,Wendy'S,"[B, A]","[A, B, A, A]"
3,Tov Kosher Kitchen,"[B, Z, A]","[Z, A, A, B]"
4,Regina Caterers,[A],"[A, A, A, A, A]"
5,Taste The Tropics Ice Cream,[A],"[A, A, A, A]"
6,Wilken'S Fine Food,[A],"[A, A, A, A, A, A]"
7,Morris Park Bake Shop,"[B, A]","[A, A, A, A, B]"
8,Brunos On The Boulevard,"[Z, A]","[Z, A, A, A]"
9,Riviera Caterer,[A],"[A, A, A, A]"


### Itération

Il est possible de définir un curseur (de même type que PL/SQL par exemple), qui va itérer sur la liste de résultats (celle-ci sera stocké sur le serveur). Cela permet de récupérer les documents un par un, ce qui est judicieux en cas de gros volume. De plus, ceux-ci sont récupérés au format `list` pure, ce qui peut simplifier la manipulation en cas de données fortement imbriquées.

- Affichage particulier des 10 premiers restaurants du *Queens* ayant un score supérieur à 50

In [157]:
cursor = db.restaurants.find(
    {"borough": "Queens", "grades.score": { "$gte":  50}},
    {"_id": 0, "name": 1, "address.street": 1},
    batch_size = 10)

In [113]:
cursor.batch_size(5)

In [158]:
pandas.DataFrame(cursor)

,address,name
0,{'street': 'Horace Harding Boulevard'},Richer'S Bakery
1,{'street': 'Bell Boulevard'},Tequilla Sunrise
2,{'street': 'Rockaway Beach Boulevard'},Rockaway Beach Inn
3,{'street': 'Broadway'},Alfonso'S Bar
4,{'street': 'Woodhaven Boulevard'},Pio Pio
...,...,...
78,{'street': '30 Avenue'},Queens Comfort Restaurant
79,{'street': 'Union Turnpike'},Koyla
80,{'street': '37 Road'},Jackson Heights Food Court
81,{'street': 'Union Street'},K & D Internet Inc


In [156]:
for c in cursor:
    print("\n\n" + ("*" * 30) + "\n")
    print(pandas.DataFrame(c))



******************************

                         address             name
street  Horace Harding Boulevard  Richer'S Bakery


******************************

               address              name
street  Bell Boulevard  Tequilla Sunrise


******************************

                         address                name
street  Rockaway Beach Boulevard  Rockaway Beach Inn


******************************

         address           name
street  Broadway  Alfonso'S Bar


******************************

                    address     name
street  Woodhaven Boulevard  Pio Pio


******************************

                   address                     name
street  Northern Boulevard  E-Dah Korean Bbq Lounge


******************************

                    address                name
street  Cross Bay Boulevard  Roma View Catering


******************************

                 address                 name
street  Roosevelt Avenue  Hornado Ecuatoriano


********

In [138]:
cursor.limit(5)

In [159]:
dir(cursor)

['_Cursor__address',
 '_Cursor__allow_disk_use',
 '_Cursor__batch_size',
 '_Cursor__check_okay_to_chain',
 '_Cursor__codec_options',
 '_Cursor__collation',
 '_Cursor__collection',
 '_Cursor__collname',
 '_Cursor__comment',
 '_Cursor__data',
 '_Cursor__dbname',
 '_Cursor__die',
 '_Cursor__empty',
 '_Cursor__exhaust',
 '_Cursor__exhaust_mgr',
 '_Cursor__explain',
 '_Cursor__explicit_session',
 '_Cursor__hint',
 '_Cursor__id',
 '_Cursor__killed',
 '_Cursor__limit',
 '_Cursor__manipulate',
 '_Cursor__max',
 '_Cursor__max_await_time_ms',
 '_Cursor__max_scan',
 '_Cursor__max_time_ms',
 '_Cursor__min',
 '_Cursor__modifiers',
 '_Cursor__ordering',
 '_Cursor__projection',
 '_Cursor__query_flags',
 '_Cursor__query_spec',
 '_Cursor__read_concern',
 '_Cursor__read_preference',
 '_Cursor__retrieved',
 '_Cursor__return_key',
 '_Cursor__send_message',
 '_Cursor__session',
 '_Cursor__set_hint',
 '_Cursor__show_record_id',
 '_Cursor__skip',
 '_Cursor__snapshot',
 '_Cursor__spec',
 '__class__',
 '__copy

Plutôt que d'avoir les documents un par un, il est ausi possible de les avoir par paquets avec la fonction `batch(n)` (ainsi que `page(n)` et `json(n)` qui diffèrent sur le type de ce qui est retourné) sur le curseur (`n` étant donc le nombre de documents renvoyés)

- Vue des différences entre les 3 opérateurs

```python
cursor = db.restaurants.iterate(limit = 15)
cursor$batch(5)
cursor$page(5)
cursor$json(5)
```

## A faire


### Rendu

Envoyez votre fichier (notebook `ipynb` - avec votre nom dans le nom du fichier) par mail à **francois-xavier.jollois@u-paris.fr**.

### Restaurants 

1. Lister tous les restaurants de la chaîne "Bareburger" (rue, quartier)
1. Lister les trois chaînes de restaurant les plus présentes
1. Donner les 10 styles de cuisine les plus présents dans la collection
1. Lister les 10 restaurants les moins bien notés (note moyenne la plus haute)
1. Lister par quartier le nombre de restaurants, le score moyen et le pourcentage moyen d'évaluation A

#### Questions complémentaires

Nécessitent une recherche sur la toile pour compléter ce qu’on a déjà vu dans ce TP.

1. Lister les restaurants (nom et rue uniquement) situés sur une rue ayant le terme “Union” dans le nom
1. Lister les restaurants ayant eu une visite le 1er février 2014
1. Lister les restaurants situés entre les longitudes -74.2 et -74.1 et les lattitudes 40.1 et 40.2


### AirBnB

Nous allons travailler sur des données AirBnB. Celles-ci sont stockées sur le serveur Mongo dans la collection `listingsAndReviews` de la base `sample_airbnb`.

> [Aide sur les données](https://docs.atlas.mongodb.com/sample-data/sample-airbnb)

1. Créer la connexion à la collection dans R
1. Donner le nombre de logements
1. Lister les informations du logement "10545725" (cf _id)
1. Lister les différentes types de logements possibles cf (room_type)
1. Donner le nombre de logements par type
1. Représenter graphiquement la distribution des prix 
1. Croiser numériquement et graphiquement le type et le prix (price)
1. Représenter la distribution du nombre d'avis (cf reviews - à calculer)
1. Croiser graphiquement le nombre d'avis et le prix, en ajoutant l'information du type de logement

In [47]:
import pandas